# Layer: Bronze (Raw/Ingestion)
**Project:** Lean Logistics Data Pipeline  
**Business Domain:** E-commerce (Olist Dataset)

---
## 📑 Notebook Information
| Version | Date | Author | Summary of Changes |
| :--- | :--- | :--- | :--- |
| v1.0 | 2026-02-20 | Tássia Marchito | Initial ingestion of CSV files to Delta tables. |
| v1.1 | 2026-02-20 | Tássia Marchito | Fixed UC `_metadata` error and added dynamic file discovery loop. |

---
## 🎯 Objectives
The Bronze layer acts as the entry point for all raw data. It preserves the state of the source system while adding governance.
* **Fidelity:** Data is stored exactly as it arrived from the source (CSV).
* **Immutability:** Historical records are preserved with ingestion timestamps.
* **Traceability:** Every row includes the source file path via Unity Catalog `_metadata`.
* **Governance:** Initial table registration in the Catalog with basic tags and primary key definitions.

In [0]:
display(dbutils.fs.ls("/Volumes/cat_tm_services_bronze/db_logistics/raw_files"))

In [0]:
%skip
%sql
-- 1. Removendo tabelas sem o prefixo tb_ no novo schema
DROP TABLE IF EXISTS cat_tm_services_bronze.db_logistics.customers;
DROP TABLE IF EXISTS cat_tm_services_bronze.db_logistics.geolocation;
DROP TABLE IF EXISTS cat_tm_services_bronze.db_logistics.order_items;
DROP TABLE IF EXISTS cat_tm_services_bronze.db_logistics.order_payments;
DROP TABLE IF EXISTS cat_tm_services_bronze.db_logistics.order_reviews;
DROP TABLE IF EXISTS cat_tm_services_bronze.db_logistics.orders;
DROP TABLE IF EXISTS cat_tm_services_bronze.db_logistics.products;
DROP TABLE IF EXISTS cat_tm_services_bronze.db_logistics.sellers;
DROP TABLE IF EXISTS cat_tm_services_bronze.db_logistics.product_category_name_translation;

-- 2. (OPCIONAL) Removendo o schema antigo 'logistics' caso ele ainda exista
-- ATENÇÃO: Use CASCADE para apagar tudo que estiver dentro dele
-- DROP SCHEMA IF EXISTS cat_tm_services_bronze.logistics CASCADE;

In [0]:
from pyspark.sql.functions import current_timestamp, col
import os

In [0]:
# 1. Configuração de Caminhos e Destino
target_schema = "cat_tm_services_bronze.db_logistics"
volume_path = "/Volumes/cat_tm_services_bronze/db_logistics/raw_files"

# 2. Dicionário de Metadados (Mapeamento de PKs e Áreas)
# Essencial para garantir a integridade e governança na Bronze
bronze_metadata = {
    "olist_customers_dataset.csv": {"pk": "customer_id", "area": "crm", "cm": "Customer information and location."},
    "olist_geolocation_dataset.csv": {"pk": "geolocation_zip_code_prefix, geolocation_lat, geolocation_lng", "area": "logistics", "cm": "Geographic coordinates for zip codes."},
    "olist_order_items_dataset.csv": {"pk": "order_id, order_item_id", "area": "sales", "cm": "Items contained in each order."},
    "olist_order_payments_dataset.csv": {"pk": "order_id, payment_sequential", "area": "finance", "cm": "Payment methods and installment details."},
    "olist_order_reviews_dataset.csv": {"pk": "review_id", "area": "crm", "cm": "Customer satisfaction reviews."},
    "olist_orders_dataset.csv": {"pk": "order_id", "area": "sales", "cm": "Core order management data."},
    "olist_products_dataset.csv": {"pk": "product_id", "area": "catalog", "cm": "Product catalog details."},
    "olist_sellers_dataset.csv": {"pk": "seller_id", "area": "partners", "cm": "Sellers information."},
    "product_category_name_translation.csv": {"pk": "product_category_name", "area": "catalog", "cm": "Translation of category names to English."}
}

# 3. Descoberta Dinâmica de Arquivos
files_in_volume = [f.name for f in dbutils.fs.ls(volume_path) if f.name.endswith(".csv")]

print(f"📦 Starting Bronze Ingestion for {len(files_in_volume)} files...\n")

for file_name in files_in_volume:
    # Padronização de Nome: olist_orders_dataset.csv -> tb_orders
    clean_name = file_name.replace("olist_", "").replace("_dataset.csv", "").replace(".csv", "")
    table_name = f"tb_{clean_name}"
    full_target_path = f"{target_schema}.{table_name}"
    full_source_path = os.path.join(volume_path, file_name)
    
    # Busca metadados ou usa padrão
    info = bronze_metadata.get(file_name, {"pk": None, "area": "general", "cm": "Raw ingested data."})
    
    print(f"🔄 Processing: {table_name}")
    
    # LEITURA: Usando _metadata conforme exigência do Unity Catalog
    df_raw = (spark.read.format("csv")
              .option("header", "true")
              .option("inferSchema", "true")
              .load(full_source_path)
              .select("*", "_metadata.file_path")
              .withColumnRenamed("file_path", "_source_file")
              # Aplicando ts_ na coluna de auditoria conforme solicitado
              .withColumn("ts_ingestion", current_timestamp()))
    
    # ESCRITA: Formato Delta com Overwrite de Schema
    df